#  Использование предобученных моделей для классификации изображений

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann 
* https://pytorch.org/vision/0.16/transforms.html#v2-api-reference-recommended
* https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html
* https://pytorch.org/vision/stable/models.html
* https://albumentations.ai/docs/getting_started/image_augmentation/
* https://www.neurotec.uni-bremen.de/drupal/node/30

## Задачи для совместного разбора

1\. Загрузите предобученную модель из `torchvision`. Познакомьтесь с ее архитектурой. Заморозьте веса нескольких слоев.

Загрузка предобученных моделей из `torchvision` — это удобный способ использовать уже обученные архитектуры для вашей задачи. 

Что нужно знать:

- torchvision предоставляет множество предобученных моделей для различных задач: классификация, обнаружение объектов, сегментация.
- при загрузке модели вы можете указать, какие именно веса использовать; для некоторых моделей есть варианты
- предобученные модели могут служить отличной отправной точкой для решения вашей задачи
- предобученную модель нужно выбирать, исходя 
    - вашей задачи
    - того, на чем она училась и
    - ваших ресурсов (мощность машины)

In [11]:
import torchvision.models as models
import torch as th

model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT )

Вы можете дообучить предобученную модель на своем наборе данных, что часто приводит к лучшим результатам, чем обучение с нуля. Это особенно полезно, если ваш набор данных мал или схож с тем, на котором модель была обучена.

Часто необходимо заменить последний полносвязный слой модели, чтобы он соответствовал числу классов в вашей задаче. Это легко сделать, просто нужно понять, как к нему обратиться.

In [2]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

Нас интересует последний блок `classifier`

In [12]:
import torch.nn as nn
num_classes = 10
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

Теперь модель будет выдавать 10 чисел вместо 1000. Но новый вариант `nn.Linear` пока инициализирован случайным образом. Чтобы прогнозы были адекватные, модель надо дообучить. Варианты:

1. Просто берем эту модель и используем в стандартном цикле обучения. Обучаться будет последний слой, но вместе с ним и все предыдущие.
2. "Заморозить" все слои, кроме последнего (нового). Тогда обучаться будет последний слой, а остальные - не будут меняться
3. Промежуточный вариант: заморозить часть верхних слоев.

Что выбрать? Обычно:
-  более глубокие слои (ближе к выходу) отвечают за более специфические признаки, связанные с конкретной задачей (например, распознавание объектов) - эти слои часто требуют дообучения.
- первые слои обычно захватывают более общие признаки (например, края и текстуры), которые могут быть полезны для множества задач - их часто замораживают, чтобы сохранить эти общие представления.

Базовые интуиции:
- чем больше данных, тем больше можно оставить размороженным;
- в зависимости от того, насколько ваша задача близка к тому, на чем училась модель изначально, вы можете заморозить больше или меньше слоев

В итоге все сводится к тому, что вы должны экспериментировать и измерять качество.


Как замораживать: нужно соответствующему весу указать `requires_grad = False`. Обычно удобно сделать условие по названию. Названия параметров вместе с самими тензорами можно получить при помощи `named_parameters`.

Рекомендую использовать именно метод `requires_grad_` (пример ниже), так как при простом присваивании может закрасться ошибка (если нет средств анализа кода)

In [13]:
list(n for n, _ in model.named_parameters())

['features.0.0.weight',
 'features.0.1.weight',
 'features.0.1.bias',
 'features.1.0.block.0.0.weight',
 'features.1.0.block.0.1.weight',
 'features.1.0.block.0.1.bias',
 'features.1.0.block.1.fc1.weight',
 'features.1.0.block.1.fc1.bias',
 'features.1.0.block.1.fc2.weight',
 'features.1.0.block.1.fc2.bias',
 'features.1.0.block.2.0.weight',
 'features.1.0.block.2.1.weight',
 'features.1.0.block.2.1.bias',
 'features.1.1.block.0.0.weight',
 'features.1.1.block.0.1.weight',
 'features.1.1.block.0.1.bias',
 'features.1.1.block.1.fc1.weight',
 'features.1.1.block.1.fc1.bias',
 'features.1.1.block.1.fc2.weight',
 'features.1.1.block.1.fc2.bias',
 'features.1.1.block.2.0.weight',
 'features.1.1.block.2.1.weight',
 'features.1.1.block.2.1.bias',
 'features.2.0.block.0.0.weight',
 'features.2.0.block.0.1.weight',
 'features.2.0.block.0.1.bias',
 'features.2.0.block.1.0.weight',
 'features.2.0.block.1.1.weight',
 'features.2.0.block.1.1.bias',
 'features.2.0.block.2.fc1.weight',
 'features.2.0

In [14]:
model.features[0][0].weight.requires_grad

True

In [15]:
# пример
for n, w in model.named_parameters():
    if not n.startswith("classifier"):
        w.requires_grad_(False)

In [16]:
model.features[0][0].weight.requires_grad

False

In [ ]:
# пример с ошибкой (рекомендую не присваивать напрямую)
for n, w in model.named_parameters():
    if not n.startswith("classifier"):
        w.required_grad = True

In [18]:
model.features[0][0].weight.requires_grad

False

При использовании предобученных моделей важно применять те же преобразования, которые были использованы при их обучении. Обычно это изменение размера (224x224 для многих моделей) и нормализация, но может быть что-то более сложное. Рекомендую отталкиваться от реализации той модели, которую вы загрузили. В `torchvision` можно достать объект для преобразования, а если берете откуда-то из другого места, то нужно читать документацию или (что бывает чаще) читать код.


In [19]:
t = models.EfficientNet_B1_Weights.IMAGENET1K_V1.transforms()
t

ImageClassification(
    crop_size=[240]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Используя реализацию из `torchvision`, cоздайте модель `vgg16` и загрузите предобученные веса `IMAGENET1K_V1`. Выведите на экран структуру модели, количество слоев и количество настраиваемых (`requires_grad==True`) параметров модели. 

- [ ] Проверено на семинаре

<p class="task" id="2"></p>

2\. Создайте датасет `CatBreeds` на основе данных из архива `cat_breeds_4.zip`. Разбейте датасет на обучающее и тестовое множество в соотношении 80 на 20%. 

К обучающему датасету примените следующее преобразование: приведите картинки к размеру 256x256, затем обрежьте по центру с размером 224х224, затем переведите изображения в тензор и нормализуйте значения интенсивности пикселей (`mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)`).

К тестовому датасету примените преобразование `VGG16_Weights.IMAGENET1K_V1.transforms`.

- [ ] Проверено на семинаре

<p class="task" id="3"></p>

3\. Заморозьте все веса модели из предыдущего задания. Замените последний слой `Linear` классификатора на новый слой, соответствующий задаче. После изменения последнего слоя выведите на экран количество настраиваемых (`requires_grad==True`) параметров модели. Решите задачу, используя модель с замороженными весами и изменнным последним слоем. 

Постройте график изменения значения функции потерь на обучающем множестве в зависимости от номера эпохи, графики изменения метрики accuracy на обучающем в зависимости от эпохи. Выведите на экран итоговое значение метрики accuracy на обучающем и тестовом множестве. 

- [ ] Проверено на семинаре

<p class="task" id="4"></p>

4\. Повторите решение предыдущей задачи, заморозив все сверточные слои, кроме последнего (слои классификатора не замораживайте). Сравните качество полученного решения и решения из предыдущей задачи, а также время, затраченное на обучения моделей. Перед началом работы создайте модель заново.

- [ ] Проверено на семинаре

<p class="task" id="5"></p>

5\. Повторите решение задачи 3, расширив обучающий набор данных при помощи преобразований из `torchvision`, изменяющих изображение (повороты, изменение интенсивности пикселей, обрезание и т.д.). При оценке модели на тестовой выборке данные преобразования применяться не должны. Решение о том, сколько и каких слоев модели будет обучаться, примите самостоятельно. Перед началом работы создайте модель заново.

- [ ] Проверено на семинаре